<a href="https://colab.research.google.com/github/drOluOla/stintagents-evals-private/blob/main/StintAgents_Evals_Realtime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# START

In [1]:
!pip uninstall stintagents-evals-private -y
!pip install git+https://github.com/drOluOla/stintagents-evals-private.git

Found existing installation: stintagents-evals-private 0.1.0
Uninstalling stintagents-evals-private-0.1.0:
  Successfully uninstalled stintagents-evals-private-0.1.0
  Cloning https://github.com/drOluOla/stintagents-evals-private.git to /tmp/pip-req-build-09_tmutc
  Running command git clone --filter=blob:none --quiet https://github.com/drOluOla/stintagents-evals-private.git /tmp/pip-req-build-09_tmutc
  Resolved https://github.com/drOluOla/stintagents-evals-private.git to commit 00844033af44a35f40509de120e4a92bfe1a78a6
  Preparing metadata (setup.py) ... done
  Created wheel for stintagents-evals-private: filename=stintagents_evals_private-0.1.0-py3-none-any.whl size=18464 sha256=80bb82317229a4c6664882d155402ee3dfb5b410788343141b31cbd3821491de
  Stored in directory: /tmp/pip-ephem-wheel-cache-mn08calg/wheels/d1/98/ec/2ac06e49defaa85badc47b0e3b5c146771bc7d6a0e58639b02
Successfully built stintagents-evals-private


In [2]:
import os
from google.colab import userdata

# Set API key in "Secrets" tab before running
openai_api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_api_key

In [3]:
import io
import wave
import json
import asyncio
import random
import threading
from logging import log
from datetime import datetime
from typing import AsyncIterator, Optional, Tuple, Dict, Any, List
from dataclasses import dataclass, field

import numpy as np
from scipy import signal

from pydub import AudioSegment

import matplotlib.pyplot as plt
import matplotlib.patches as patches

import openai
from openai import OpenAI, AsyncOpenAI

import gradio as gr

from agents import Agent, Runner, function_tool, SQLiteSession
from agents.extensions.handoff_prompt import prompt_with_handoff_instructions

from inspect_ai import Task, eval, task
from inspect_ai.dataset import Sample, Dataset, MemoryDataset
from inspect_ai.scorer import Score, scorer, mean, accuracy, model_graded_fact, match, includes
from inspect_ai.solver import generate, system_message, TaskState, solver, Generate
from inspect_ai.model import get_model

import torch

import stintagents.config as config
from stintagents import get_or_create_event_loop, create_gradio_interface
from stintagents.config import CONVERSATION_SESSIONS, CURRENT_TOOL_EXPECTED


# To Handle event loop conflict in colab
get_or_create_event_loop()

[init] Realtime API ready!


<_UnixSelectorEventLoop running=True closed=False debug=False>

In [4]:
from stintagents.config import set_agent_personas

# Define your custom personas
custom_personas = {
      "HR Manager": {
          "voice": "alloy",
          "speed": 1.0,
          "description": "Professional & Welcoming",
          "emoji": "🧔🏻‍♀️",
          "color": "#45B7B8",
          "specialty": "HR Policies & Benefits"
      },
      "AI Colleague": {
          "voice": "marin",
          "speed": 1.0,
          "description": "Friendly & Collaborative",
          "emoji": "👧🏾",
          "color": "#45B7B8",
          "specialty": "General Workplace Support"
      },
      "IT Staff": {
          "voice": "sage",
          "speed": 1.1,
          "description": "Technical & Helpful",
          "emoji": "👩🏽",
          "color": "#45B7B8",
          "specialty": "IT Setup & Support"
      },
      "Line Manager": {
          "voice": "echo",
          "speed": 1.0,
          "description": "Supportive & Strategic",
          "emoji": "🧔🏿",
          "color": "#45B7B8",
          "specialty": "Team Leadership & Goals"
      }

}

# Apply your changes
set_agent_personas(custom_personas)

In [5]:
@function_tool
async def get_welcome_info() -> str:
    """Welcome the new employee to the team."""
    welcome_message = (
        'Hello there! Welcome to our organisation. We are very thrilled to have you onboard! '
        'You are joining a group of passionate, supportive people who are here to help you succeed. '
        'Feel free to ask me anything about getting started, company culture, or where to find things.'
    )
    CURRENT_TOOL_EXPECTED["expected"] = welcome_message
    return welcome_message

@function_tool
async def get_benefits_info(benefit_type: str) -> str:
    """Get information about employee benefits like health, vacation, retirement, learning, wellness, and remote work policies."""
    benefits_info = {
        'health': 'Our comprehensive health insurance covers medical, dental, and vision with 90% coverage. Open enrollment is in November.',
        'vacation': 'You start with 15 vacation days, 10 sick days, and 12 company holidays. Vacation accrues monthly.',
        'retirement': 'We offer a 401k with 6% company match (fully vested after 2 years). Financial planning resources available.',
        'learning': 'Annual $2,500 learning budget for courses, conferences, and certifications. Internal mentorship program available.',
        'wellness': 'On-site gym, wellness stipend of $100/month, mental health support through EAP program.',
        'remote': 'Flexible hybrid work - 2 days in office required. Full remote work considered case-by-case.'
    }
    benefit_lower = benefit_type.lower()
    for key, info in benefits_info.items():
        if key in benefit_lower:
            CURRENT_TOOL_EXPECTED["expected"] = info
            return info
    fallback = 'Our main benefits include health insurance, retirement planning, learning budget, and wellness programs.'
    CURRENT_TOOL_EXPECTED["expected"] = fallback
    return fallback

@function_tool
async def get_workplace_info(info_type: str) -> str:
    """Get information about workplace facilities, culture, work-life balance, parking, and dress code."""
    workplace_info = {
        'kitchen': 'Our 3rd floor kitchen has free coffee, tea, and snacks plus microwave and fridge.',
        'culture': 'Our culture is collaborative and welcoming - we value teamwork and flexibility. Friday happy hours, monthly team building, and quarterly volunteer days.',
        'worklife': 'Work-life balance is a priority here. Flexible start times (8-10am), unlimited PTO, and we respect personal time - no emails after 6pm or weekends unless urgent.',
        'facilities': 'We have a free gym in the basement, quiet rooms on 4th floor, phone booths for calls, and a rooftop terrace for breaks.',
        'parking': 'Free parking in building garage (B1-B3), bike storage in B1, and Red line Metro is 2 blocks away.',
        'dress': 'Smart casual dress code - jeans are fine, just avoid shorts/flip-flops. Fridays are even more relaxed.'
    }
    info_lower = info_type.lower()
    for key, info in workplace_info.items():
        if key in info_lower or any(word in info_lower for word in key.split()):
            CURRENT_TOOL_EXPECTED["expected"] = info
            return info
    fallback = 'Feel free to ask about our kitchen, company culture, work-life balance, facilities, parking, or dress code.'
    CURRENT_TOOL_EXPECTED["expected"] = fallback
    return fallback

@function_tool
async def get_it_setup_info(setup_type: str) -> str:
    """Get information about IT setup including laptop, accounts, software, VPN, phone, and security."""
    it_setups = {
        'laptop': 'Youll receive a MacBook Pro M3 or Dell XPS (your choice). Setup takes 1-2 days with all necessary software pre-configured.',
        'accounts': 'Ill create your email, Slack, GitHub, and system accounts today. Youll get temporary passwords to change on first login.',
        'software': 'Standard setup includes Office 365, Slack, Zoom, VS Code, and role-specific tools.',
        'vpn': 'VPN access will be configured for secure remote work. Ill show you how to connect and troubleshoot common issues.',
        'phone': 'Company phone available if needed for your role. BYOD policy allows using personal devices with MDM enrollment.',
        'security': 'Youll need to complete security training and set up 2FA on all accounts. Ill walk you through the security protocols.'
    }
    setup_lower = setup_type.lower()
    for key, info in it_setups.items():
        if key in setup_lower:
            CURRENT_TOOL_EXPECTED["expected"] = info
            return info
    fallback = 'Your IT package includes laptop, accounts, software, and security setup.'
    CURRENT_TOOL_EXPECTED["expected"] = fallback
    return fallback

@function_tool
async def get_team_info(info_type: str) -> str:
    """Get information about team structure, first task, meetings, and communication channels."""
    team_info = {
        'first_task': 'Your first task will be completing our Standard Operating Procedure training module. After reviewing the materials, youll need to answer a brief assessment to confirm your understanding.',
        'structure': 'Our team has 8 members: 3 senior developers, 2 product managers, 2 designers, and you! We work in cross-functional squads.',
        'meetings': 'Daily standups at 9:30 AM, sprint planning every 2 weeks, and monthly all-hands. Ill add you to all relevant calendars.',
        'culture': 'We value collaboration, learning, and work-life balance. Monthly team lunches and quarterly offsite events.',
        'communication': 'Slack for daily chat, email for formal communications, Zoom for meetings. #general and #team-updates are key channels.'
    }
    info_lower = info_type.lower()
    for key, info in team_info.items():
        if key in info_lower:
            CURRENT_TOOL_EXPECTED["expected"] = info
            return info
    fallback = 'Our team is collaborative and supportive. Well have a proper 1-on-1 this week to discuss your role in detail.'
    CURRENT_TOOL_EXPECTED["expected"] = fallback
    return fallback

@function_tool
async def access_code_tool(request_type: str) -> str:
    """Get access codes for VPN or building pass."""
    codes = {
        'vpn': 'VPN-000123',
        'pass': 'PASS-ABC456'
    }
    request_lower = request_type.lower()
    if 'vpn' in request_lower:
        result = f'VPN access code: {codes["vpn"]}'
        CURRENT_TOOL_EXPECTED["expected"] = result
        return result
    elif 'pass' in request_lower:
        result = f'Pass: {codes["pass"]}'
        CURRENT_TOOL_EXPECTED["expected"] = result
        return result
    fallback = f'VPN: {codes["vpn"]}, PASS: {codes["pass"]}'
    CURRENT_TOOL_EXPECTED["expected"] = fallback
    return fallback


In [6]:
from agents.realtime import RealtimeAgent, realtime_handoff

# Define Realtime Agents for multi-agent onboarding
ai_colleague_realtime = RealtimeAgent(
    name="AI Colleague",
    handoff_description="A friendly colleague who can answer questions about workplace culture, facilities, and daily work life.",
    instructions="""You are an AI Colleague in a live video conference onboarding call with a new employee.
    You are one of several specialists on this call (HR Manager, IT Staff, Line Manager).

    # Your Role
    You provide enthusiastic, warm support about workplace culture and daily life at the company.

    # When You Receive a Handoff
    You will receive context about what the user asked. DO NOT introduce yourself or say hello again.
    Instead, respond DIRECTLY to their question using your tools.

    # Routine
    1. Use the get_workplace_info tool immediately to answer their question.
    2. Provide the information warmly and encouragingly.

    # Delegation - Transfer to Other Specialists When Asked About:
    - HR policies, benefits, access codes → transfer to HR Manager
    - Technical setup, software, accounts, laptops, VPN → transfer to IT Staff
    - First task, team structure, role, meetings → transfer to Line Manager

    This is a live video call - respond immediately and directly to their question.""",
    tools=[get_workplace_info]
)

it_staff_realtime = RealtimeAgent(
    name="IT Staff",
    handoff_description="An IT specialist who can help with technical setup, equipment, accounts, and software.",
    instructions="""You are an IT Staff member in a live video conference onboarding call with a new employee.
    You are one of several specialists on this call (HR Manager, Line Manager, AI Colleague).

    # Your Role
    You help new employees get their technical setup completed.

    # When You Receive a Handoff
    You will receive context about what the user asked. DO NOT introduce yourself or say hello again.
    Instead, respond DIRECTLY to their technical question using your tools.

    # Routine
    1. Use the get_it_setup_info tool immediately to answer their question.
    2. Explain things clearly and helpfully.

    # Delegation - Transfer to Other Specialists When Asked About:
    - HR policies, benefits, access codes → transfer to HR Manager
    - Workplace culture, facilities, kitchen, parking → transfer to AI Colleague
    - First task, team structure, role, meetings → transfer to Line Manager

    This is a live video call - respond immediately and directly to their question.""",
    tools=[get_it_setup_info]
)

line_manager_realtime = RealtimeAgent(
    name="Line Manager",
    handoff_description="A line manager who can provide information about the team, first task, and role guidance.",
    instructions="""You are a Line Manager in a live video conference onboarding call with a new employee.
    You are one of several specialists on this call (HR Manager, IT Staff, AI Colleague).

    # Your Role
    You help new employees understand their role, team, and first tasks.

    # When You Receive a Handoff
    You will receive context about what the user asked. DO NOT introduce yourself or say hello again.
    Instead, respond DIRECTLY to their question about the role or team using your tools.

    # Routine
    1. Use the get_team_info tool immediately to answer their question.
    2. Be enthusiastic and supportive about their new role.

    # Delegation - Transfer to Other Specialists When Asked About:
    - HR policies, benefits, access codes → transfer to HR Manager
    - Technical setup, software, accounts, laptops, VPN → transfer to IT Staff
    - Workplace culture, facilities, kitchen, parking → transfer to AI Colleague

    This is a live video call - respond immediately and directly to their question.""",
    tools=[get_team_info]
)

hr_manager_realtime = RealtimeAgent(
    name="HR Manager",
    handoff_description="An HR Manager who can welcome new employees and provide information about benefits and policies.",
    instructions="""You are an HR Manager coordinating a live video conference onboarding call with a new employee.
    Other specialists (IT Staff, Line Manager, AI Colleague) are present on this call.

    # Your Role
    You welcome employees, provide HR information, and coordinate with specialists.

    # When You Receive a Handoff
    If you receive a handoff from another specialist, respond directly to what the user asked.
    DO NOT re-introduce yourself - just answer their question.

    # Routine for Initial Greeting (start of call only)
    1. Use the get_welcome_info tool to welcome them warmly.
    2. Ask how you can help them get started.

    # Routine for Benefits/Policy Questions
    1. Use the get_benefits_info or access_code_tool to provide accurate information.

    # Delegation - Transfer to Other Specialists When Asked About:
    - Technical setup, software, accounts, laptops, VPN → transfer to IT Staff
    - First task, team structure, role, meetings → transfer to Line Manager
    - Workplace culture, facilities, kitchen, parking, dress code → transfer to AI Colleague

    If they ask something off-topic, say: "I'm not sure I followed—would you mind putting that differently?"

    This is a live video call - respond immediately and naturally.""",
    tools=[get_welcome_info, get_benefits_info, access_code_tool],
    handoffs=[
        realtime_handoff(it_staff_realtime),
        realtime_handoff(line_manager_realtime),
        realtime_handoff(ai_colleague_realtime)
    ]
)

# Setup full multi-directional handoffs - any agent can transfer to any other agent
ai_colleague_realtime.handoffs = [
    realtime_handoff(hr_manager_realtime),
    realtime_handoff(it_staff_realtime),
    realtime_handoff(line_manager_realtime)
]

it_staff_realtime.handoffs = [
    realtime_handoff(hr_manager_realtime),
    realtime_handoff(ai_colleague_realtime),
    realtime_handoff(line_manager_realtime)
]

line_manager_realtime.handoffs = [
    realtime_handoff(hr_manager_realtime),
    realtime_handoff(it_staff_realtime),
    realtime_handoff(ai_colleague_realtime)
]

In [ ]:
# ╔══════════════════════════════════════════════════════════════════╗
# ║              REALTIME API SETUP (Official SDK)                   ║
# ║        Uses OpenAI RealtimeAgent for low-latency speech          ║
# ╚══════════════════════════════════════════════════════════════════╝

# Initialize a specific conversation session with ID "session_123"
conversation_id = "session_123"
CONVERSATION_SESSIONS[conversation_id] = SQLiteSession(conversation_id)

print("✓ Session initialized:", conversation_id)
print("✓ Starting agent: HR Manager")
print("✓ Multi-agent handoffs enabled: IT Staff, Line Manager, AI Colleague")

# Create the Gradio interface with Realtime Agent
# Note: We pass the hr_manager_realtime agent directly
iface = create_gradio_interface(CONVERSATION_SESSIONS, conversation_id, hr_manager_realtime)
iface.launch(share=True, debug=True)


✓ Session initialized: session_123
✓ Starting agent: HR Manager
✓ Multi-agent handoffs enabled: IT Staff, Line Manager, AI Colleague
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3daaa9b5b0ae5d52ec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[INFO] Creating new persistent session for conversation session_123
[INFO] Processing 60000 samples with session session_123
[INFO] Agent started: HR Manager
[TOOL] HR Manager calling: get_welcome_info with args: {}
[TOOL] get_welcome_info result: Hello there! Welcome to our organisation. We are very thrilled to have you onboard! You are joining a group of passionate, supportive people who are here to help you succeed. Feel free to ask me anyth
[INFO] Agent started: HR Manager
[WARN] No transcript captured for HR Manager's response
[INFO] Processing 72000 samples with session session_123
[TRANSCRIPT] User said: Hello. Hi, everyone.
[TRANSCRIPT] HR Manager said: Hello there! Welcome to our organization. We are very thrilled to have you onboard! You are joining a group of passionate, supportive people who are here to help you succeed. How can I help you get started?
[INFO] Agent started: HR Manager
[INFO] Handoff initiated: HR Manager → Line Manager, waiting for pending transcripts...
[D